# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create city dataframe
city500_weather_data_df = pd.read_csv('../output_data/city500_weather_data.csv')
city500_weather_data_df.dropna(inplace = True) 
city500_weather_data_df.head()

,city_name,country_code,lattitude,longitude,temp,wind_speed,cloudiness,humidity
0,bluff,nz,-76.039276,172.583202,50.00,10.00,100,87
1,aykhal,ru,68.293438,110.538752,-35.50,0.89,6,76
2,mataura,pf,-64.090568,-161.013526,50.00,10.00,70,87
3,puerto ayora,ec,3.038544,-89.474206,82.83,18.41,40,70
4,vaini,to,-35.109108,-179.183464,73.40,13.80,75,94


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city500_weather_data_df[["lattitude", "longitude"]]

# Fill NaN values and convert to float
rating = city500_weather_data_df["humidity"].astype(float)


In [84]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_city_df= city500_weather_data_df.query('temp < 80 and temp > 70 and wind_speed < 20 and cloudiness == 0')
ideal_city_df

,city_name,country_code,lattitude,longitude,temp,wind_speed,cloudiness,humidity
18,carnarvon,au,-20.437623,104.318186,71.60,18.41,0,88
45,bonthe,sl,5.104690,-15.832076,78.84,8.10,0,82
86,georgetown,sh,-15.850903,-19.886925,77.13,12.66,0,76
136,pisco,pe,-26.218014,-98.036415,77.00,9.22,0,61
156,tessalit,ml,22.107638,-0.843256,70.20,9.69,0,20
158,atar,mr,22.787544,-12.266497,75.20,10.45,0,16
186,cockburn town,bs,26.843429,-73.961328,75.69,7.49,0,69
188,sao filipe,cv,10.084995,-26.290857,75.74,4.07,0,67
221,taoudenni,ml,23.855242,-8.860417,72.59,12.30,0,22
232,pathein,mm,17.319920,93.046659,70.20,6.38,0,83


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
ideal_city_df['hotel_name'] = ""
ideal_city_df['hotel_address'] = ""
ideal_city_df.head()

,city_name,country_code,lattitude,longitude,temp,wind_speed,cloudiness,humidity,hotel_name,hotel_address
18,carnarvon,au,-20.437623,104.318186,71.60,18.41,0,88,,
45,bonthe,sl,5.104690,-15.832076,78.84,8.10,0,82,,
86,georgetown,sh,-15.850903,-19.886925,77.13,12.66,0,76,,
136,pisco,pe,-26.218014,-98.036415,77.00,9.22,0,61,,
156,tessalit,ml,22.107638,-0.843256,70.20,9.69,0,20,,


In [22]:

#locations = ideal_city_df[["lattitude", "longitude"]]
# find the closest restaurant of each type to coordinates
for index, row in ideal_city_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    location= str(row['lattitude']) + ',' + str(row['longitude'])
    #location = zip(row['lattitude'], row['longitude'])
    print(location)
    params = {
        "location": location, 
        "radius":8000,
        "type": "lodging",
        "key": g_key,
        }
    response = requests.get(base_url, params=params).json()
    results = response['results']

    
    try:
        print(f"Closest {row['city_name']} hotel is {results[0]['name']}.")
        
        ideal_city_df.loc[index, 'hotel_name'] = results[0]['name']
        ideal_city_df.loc[index, 'hotel_address'] = results[0]['vicinity']
        
        
    except (KeyError, IndexError):
        print(f"Missing field/result... skipping. ")      
    
      

-20.43762295852619,104.31818619404476
Missing field/result... skipping. 
5.104689557347328,-15.832075838655756
Missing field/result... skipping. 
-15.850903259369106,-19.886924938836813
Missing field/result... skipping. 
-26.2180142311936,-98.03641539938229
Missing field/result... skipping. 
22.10763826397725,-0.8432557925806066
Missing field/result... skipping. 
22.787544482639827,-12.266496642890049
Missing field/result... skipping. 
26.84342886365968,-73.96132843539212
Missing field/result... skipping. 
10.084995278038406,-26.29085728593512
Missing field/result... skipping. 
23.85524236747516,-8.860417418445621
Missing field/result... skipping. 
17.319920105982813,93.0466591232548
Missing field/result... skipping. 
12.753849005483033,60.64000863413622
Missing field/result... skipping. 
-21.093837581588744,-40.976551899005436
Closest marataizes hotel is Pousada do Rosalvo.
-13.823825348480526,45.10019588566226
Missing field/result... skipping. 
-47.31245983529988,-53.735357346608936


In [41]:
# Convert hotel_name  to a list
# Convert city name  to list
hotel_name = ideal_city_df["hotel_name"].tolist()
city_name= ideal_city_df["city_name"].tolist()
country_name = ideal_city_df["country_code"].tolist()
latitude = ideal_city_df["lattitude"].tolist()
longitude = ideal_city_df["longitude"].tolist()

address = []
for i in range(len(hotel_name)):
   # if isinstance(i, str) :
        if len(hotel_name[i]) > 0:
            address.append({
            "hotel name" : hotel_name[i],
            "city" : city_name[i],
            "country" : country_name[i],
            "latitude" : latitude[i],
            "longitude" : longitude[i]
        })
 


In [67]:

selected_city = pd.DataFrame.from_dict(address)
selected_city[['latitude', 'longitude']]
selected_city

,hotel name,city,country,latitude,longitude
0,Pousada do Rosalvo,marataizes,br,-21.093838,-40.976552
1,Selous Kinga Lodge,utete,tz,-7.876731,38.433204


In [79]:
address[0:5]

[{'hotel name': 'Pousada do Rosalvo',
  'city': 'marataizes',
  'country': 'br',
  'latitude': -21.093837581588744,
  'longitude': -40.976551899005436},
 {'hotel name': 'Selous Kinga Lodge',
  'city': 'utete',
  'country': 'tz',
  'latitude': -7.876731142702526,
  'longitude': 38.43320437435671}]

In [81]:
# Create a map using state centroid coordinates to set markers
marker_locations =selected_city[['latitude', 'longitude']]

info_box_template = """
<dl>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""

info_box_template.format(**{"city":"Lima","country":"Peru"})

'\n<dl>\n<dt>City</dt><dd>Lima</dd>\n<dt>Country</dt><dd>Peru</dd>\n</dl>\n'

In [80]:
# Create a marker_layer using the selected_city_db to fill the info box
fig = gmaps.figure()


markers = gmaps.marker_layer(marker_locations,
    info_box_content=[info_box_template.format(**locale) for locale in address])
                            
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))